### Add venv modules

In [1]:
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from pathlib import Path
import pandas as pd
import pint
import sys
import pyvista as pv
from tqdm import tqdm
import logging
import ast
from joblib import dump, load

In [2]:
sys.path.append(str(Path.cwd().parent))

### Add local modules

In [ ]:
from scr.utils import safe_parse_quantity , delete_comsol_fields
from scr.comsol_module.comsol_classes import COMSOL_VTU
from scr.comsol_module.entropy import calculate_S_therm

### Import 

In [15]:
ROOT = Path().cwd().parent
PARAMETER_SPACE = "03"
DATA_TYPE = "Training"
IS_LOAD_NPY : bool = False
IS_LOAD_JOBLIB : bool = True
assert IS_LOAD_NPY != IS_LOAD_JOBLIB
ROOT

PosixPath('/Users/thomassimader/Documents/NIRB')

In [14]:
export_folder = ROOT / "data" / PARAMETER_SPACE / "Training_Original" # "TrainingMapped" / "s50_50_50_b50_3950_50_4950_-3950_0"
assert export_folder.exists()
import_folder = export_folder #ROOT / "data" / PARAMETER_SPACE / ""
assert import_folder.exists()


In [17]:
if IS_LOAD_NPY:
    temperatures      = np.load(import_folder  / f"{DATA_TYPE}_temperatures.npy" )
    temperatures_diff = np.load(import_folder  / f"{DATA_TYPE}_temperatures_diff.npy" )
    sim_times_path = import_folder  / f"{DATA_TYPE}_sim_times.npy"
    sim_times = np.load(sim_times_path)
if IS_LOAD_JOBLIB:
    temperatures      = load(import_folder / f"{DATA_TYPE}_temperatures.joblib" )
    temperatures_diff = load(import_folder / f"{DATA_TYPE}_temperatures_diff.joblib" )

In [18]:
comsol_data  = COMSOL_VTU( ROOT / "data" / PARAMETER_SPACE / "Training_Original" / f"{DATA_TYPE}_000.vtu")
# comsol_data  = COMSOL_VTU(import_folder / "Training_000_s50_50_50_b50_3950_50_4950_-3950_0.vti")


## Plot

### Min max temperature difference

In [19]:
N_SNAPS = len(temperatures_diff)
colors = px.colors.sample_colorscale("jet", [n/(N_SNAPS) for n in range(N_SNAPS)])

fig = go.Figure()
for idx_snap, temperature_array in enumerate(temperatures_diff):
# customdata=np.stack(([t_h] * len(comsol_data.times), [host_k] * len(comsol_data.times)) , axis=-1),
    input_args = {'x' : list(comsol_data.times.values()),
                #   'customdata' : customdata,
                #   'hovertemplate' : "Time: %{x:.2e}<br>Temperature: %{y:.2f}<br>T_h: %{customdata[0]:.2f}<br><br>host_k: %{customdata[1]:.2e}<br>",
                    'legendgroup' : f"{idx_snap:03d}",
                    'name' : f"{idx_snap:03d}",
                    'line' : dict(color=colors[idx_snap]),
                    'opacity': 0.4}
    fig.add_trace(go.Scatter(y=np.max(temperature_array, axis=1), **input_args)) 
    fig.add_trace(go.Scatter(y=np.min(temperature_array, axis=1), **input_args)) 

fig.update_layout(
    xaxis_tickformat='.2e',  # Format x-axis numbers to 2 decimal places
    yaxis_tickformat='.2f',  # Format y-axis numbers to 2 decimal places
    showlegend=False,        # Hide the legend
    title_text=f'Temperature Difference (T-T0) - {DATA_TYPE} Snapshots - Parameter Space {PARAMETER_SPACE}', # title of plot
    xaxis_title="Time [s]",    # Optional: label for x-axis
    yaxis_title="Temperature [K]"     # Optional: label for y-axis
)
fig.write_image(export_folder / f"{DATA_TYPE}_MinMaxTemperatures.png")
fig.show()


### Simulation Time

In [21]:
fig = go.Figure(data=[go.Histogram(
    x=sim_times / 60,
    histnorm='percent',
    name='control', # name used in legend and hover labels
    xbins=dict( # bins used for histogram
        # start=-4.0,
        # end=3.0,
        size=2,
        ),
    marker_color='#EB89B5',
    opacity=0.75,
     texttemplate="%{x}"
    )])

fig.update_layout(
    title_text=f'Simulation Time - {DATA_TYPE} Snapshots - Parameter Space {PARAMETER_SPACE}', # title of plot
    xaxis_title_text='Time [min]', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargroupgap=0.1, # gap between bars of the same location coordinates
)
fig.write_image(export_folder / f"{DATA_TYPE}_SimulationTime.png")
fig.show()


## Calculate Entropy


In [29]:
IS_LOAD_VTU = True

In [21]:
param_folder = ROOT / "data" / PARAMETER_SPACE / "Exports"
param_files = sorted([path for path in param_folder.rglob(f"{DATA_TYPE}*.csv")])
param_files

[PosixPath('/Users/thomassimader/Documents/NIRB/data/03/Exports/Training_000_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/data/03/Exports/Training_001_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/data/03/Exports/Training_002_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/data/03/Exports/Training_003_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/data/03/Exports/Training_004_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/data/03/Exports/Training_005_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/data/03/Exports/Training_006_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/data/03/Exports/Training_007_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/data/03/Exports/Training_008_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/data/03/Exports/Training_009_parameters.csv'),
 PosixPath('/Users/thomassimader/Documents/NIRB/da

In [22]:
df = pd.read_csv(param_files[0], index_col = 0)
df.columns

Index(['quantity'], dtype='object')

In [23]:
ureg = pint.UnitRegistry()
df['quantity_pint'] = df[df.columns[-1]].apply(lambda x : safe_parse_quantity(x, ureg))
df

,quantity,quantity_pint
aspect_ratio_dip,1.0 dimensionless,1.0 dimensionless
aspect_ratio_strike,1.25 dimensionless,1.25 dimensionless
beta,0.00059 / kelvin,0.00059 / kelvin
dip,1.1135200627723822 radian,1.1135200627723822 radian
fault_Cp,800.0 meter ** 2 / kelvin / second ** 2,800.0 meter ** 2 / kelvin / second ** 2
fault_k_long,1.5e-13 meter ** 2,1.5e-13 meter ** 2
fault_k_trans,1.5e-13 meter ** 2,1.5e-13 meter ** 2
fault_lambda,2.0 kilogram * meter / kelvin / second ** 3,2.0 kilogram * meter / kelvin / second ** 3
fault_phi,0.3 dimensionless,0.3 dimensionless
fault_rho,2500.0 kilogram / meter ** 3,2500.0 kilogram / meter ** 3


In [24]:
lambda_therm = (1 - df.loc['host_phi', "quantity_pint"]).magnitude * df.loc['host_lambda', "quantity_pint"].magnitude + \
                df.loc['host_phi', "quantity_pint"].magnitude * (4.2)
T0 = 0.5 * (df.loc["T_h", "quantity_pint"] + (20 + 273.15) * ureg.kelvin)
delta_T = (df.loc['T_h', "quantity_pint"] - (20 + 273.15) * ureg.kelvin)
print(f"{lambda_therm=}")
print(f"{T0=}")
print(f"{delta_T=}")

lambda_therm=2.0022
T0=<Quantity(393.15, 'kelvin')>
delta_T=<Quantity(200.0, 'kelvin')>


In [27]:
temperatures[0]

array([[493.15      , 493.15      , 493.15      , ..., 317.55592015,
        293.15      , 293.15      ],
       [493.15      , 493.15      , 493.15      , ..., 319.10988491,
        293.15      , 293.15      ],
       [493.15      , 493.15      , 493.15      , ..., 320.25638182,
        293.15      , 293.15      ],
       ...,
       [493.15      , 493.15      , 493.15      , ..., 320.69927982,
        293.15      , 293.15      ],
       [493.15      , 493.15      , 493.15      , ..., 320.69927981,
        293.15      , 293.15      ],
       [493.15      , 493.15      , 493.15      , ..., 320.69927981,
        293.15      , 293.15      ]], shape=(36, 19505))

In [ ]:
comsol_data_ref  = COMSOL_VTU( ROOT / "data" / PARAMETER_SPACE / "Training_Original" / f"{DATA_TYPE}_000.vtu")
comsol_data  = comsol_data_ref #COMSOL_VTU(import_folder / "Training_000_s50_50_50_b50_3950_50_4950_-3950_0.vti")

if not IS_LOAD_VTU:
    N_SNAPS = len(temperatures_diff)
    N_TIME_STEPS = len(comsol_data_ref.times)
    entropy_gen_unit = np.zeros((N_SNAPS, N_TIME_STEPS))
    entropy_gen_number = np.zeros_like(entropy_gen_unit)
    comsol_data.mesh.clear_point_data()

    for idx_snap in tqdm(range(N_SNAPS), total = N_SNAPS):
        for idx_time in range(N_TIME_STEPS):
            comsol_data.mesh.point_data["temp_field"] = temperatures[idx_snap, idx_time, :]
            cell_mesh = comsol_data.mesh.point_data_to_cell_data()
            cell_mesh.cell_data.keys()
            temp_grad = cell_mesh.compute_derivative("temp_field").cell_data["gradient"]
            s0 = calculate_S_therm(lambda_therm,
                            T0.magnitude,
                            temp_grad)
            s0_total = np.sum(s0 * comsol_data.mesh.compute_cell_sizes()["Volume"])
            L = comsol_data.mesh.bounds.z_max - comsol_data.mesh.bounds.z_min
            s0_characteristic = (lambda_therm * delta_T.magnitude**2) / (L**2 * T0.magnitude**2)
            entropy_number = s0_total / s0_characteristic / comsol_data.mesh.volume 
            entropy_gen_unit[idx_snap, idx_time] = s0_total
            entropy_gen_number[idx_snap, idx_time] = entropy_number

  1%|          | 1/100 [00:00<01:24,  1.17it/s]


ValueError: Invalid array shape. Array 'temp_field' has length (19610) but a length of (19505) was expected.

In [31]:
if IS_LOAD_VTU:
    vtu_files = sorted([path for path in import_folder.iterdir() if (path.suffix in [".vtu"] and DATA_TYPE.lower() in path.stem.lower())])
    N_SNAPS = len(vtu_files)
    N_TIME_STEPS = len(COMSOL_VTU( vtu_files[0]).times)
    entropy_gen_unit = np.zeros((N_SNAPS, N_TIME_STEPS))
    entropy_gen_number = np.zeros_like(entropy_gen_unit)
    comsol_data.mesh.clear_point_data()

    for idx_snap, vtu_file in tqdm(enumerate(vtu_files), total = len(vtu_files)): # TODO: Include TQDM
            comsol_data = COMSOL_VTU(vtu_file)
            comsol_data = delete_comsol_fields(comsol_data, fields_2_keep=["Temperature"])
            cell_mesh = comsol_data.mesh.point_data_to_cell_data()
            for idx_time, time_key in enumerate(comsol_data.times.keys()):
                field_name = comsol_data.format_field(field_name="Temperature", time=time_key)
                temp_grad = cell_mesh.compute_derivative(field_name).cell_data["gradient"]
                s0 = calculate_S_therm(lambda_therm,
                                    T0.magnitude,
                                    temp_grad)
                s0_total = np.sum(s0 * comsol_data.mesh.compute_cell_sizes()["Volume"])
                L = comsol_data.mesh.bounds.z_max - comsol_data.mesh.bounds.z_min
                s0_characteristic = (lambda_therm * delta_T.magnitude**2) / (L**2 * T0.magnitude**2)
                entropy_number = s0_total / s0_characteristic / comsol_data.mesh.volume 
                entropy_gen_unit[idx_snap, idx_time] = s0_total
                entropy_gen_number[idx_snap, idx_time] = entropy_number

100%|██████████| 100/100 [01:50<00:00,  1.11s/it]


In [32]:
np.save(export_folder / "entropy_gen_unit.npy", entropy_gen_unit)
np.save(export_folder / "entropy_gen_number.npy", entropy_gen_number)

### Plot entropy generation

In [33]:
comsol_data_ref  = COMSOL_VTU( ROOT / "data" / PARAMETER_SPACE / "Training_Original" / f"{DATA_TYPE}_000.vtu")
fig = go.Figure()

for idx_snap, entropies in enumerate(entropy_gen_unit):
    fig.add_trace(go.Scatter(x=list(comsol_data_ref.times.values()),
                             y=entropies,
                                mode='lines',
                                name=f"{idx_snap:03d}",
                                opacity=0.4,
                            line=dict(color=colors[idx_snap])
                ))
    
fig.update_layout(
    xaxis_tickformat='.2e',  # Format x-axis numbers to 2 decimal places
    yaxis_tickformat='.2f',  # Format y-axis numbers to 2 decimal places
    showlegend=False,        # Hide the legend
    title=f"Entropy generation - {DATA_TYPE} Snapshots - Parameter Space {PARAMETER_SPACE}",       # Optional: add a title
    xaxis_title="Time [s]",    # Optional: label for x-axis
    yaxis_title="Entropy [W/K]"     # Optional: label for y-axis
)

fig.write_image(export_folder / f"{DATA_TYPE}_EntropyGeneration.png")
fig.show()

### Plot entropy generation number

In [34]:
comsol_data_ref  = COMSOL_VTU( ROOT / "data" / PARAMETER_SPACE / "Training_Original" / f"{DATA_TYPE}_000.vtu")


fig = go.Figure()

for idx_snap, entropies in enumerate(entropy_gen_number):
    fig.add_trace(go.Scatter(x=list(comsol_data_ref.times.values()),
                             y=entropies,
                                mode='lines',
                                name=f"{idx_snap:03d}",
                                opacity=0.4,
                            line=dict(color=colors[idx_snap])
                ))
    
fig.update_layout(
    xaxis_tickformat='.2e',  # Format x-axis numbers to 2 decimal places
    yaxis_tickformat='.2f',  # Format y-axis numbers to 2 decimal places
    showlegend=False,        # Hide the legend
    title=f"Entropy generation number - {DATA_TYPE} Snapshots - Parameter Space {PARAMETER_SPACE}",       # Optional: add a title
    xaxis_title="Time [s]",    # Optional: label for x-axis
    yaxis_title="Entropy [-]"     # Optional: label for y-axis
)

fig.write_image(export_folder/ f"{DATA_TYPE}_EntropyGenerationNumber.png")
fig.show()

## Difference Plots